In [ ]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()

In [ ]:
using WebIO
WebIO.install_jupyter_nbextension()

In [ ]:
using Interact
using Parameters
using Plots
theme(:wong)

In [ ]:
@with_kw struct ThreeBodySystem
    m0::Float64
    m1::Float64
    m2::Float64
    m3::Float64
end
@with_kw struct Invariants
    σ1::Float64
    σ2::Float64
    σ3::Float64
end

In [ ]:
sumsq(tbs::ThreeBodySystem) = tbs.m0^2+tbs.m1^2+tbs.m2^2+tbs.m3^2
function Invariants(tbs::ThreeBodySystem;σ1=-1.0,σ2=-1.0,σ3=-1.0)
    sign(σ1)+sign(σ2)+sign(σ3)!=1 && error("the method works with TWO invariants given")
    σ3 < 0 && return Invariants(σ1=σ1,σ2=σ2,σ3=sumsq(tbs)-σ1-σ2)
    σ1 < 0 && return Invariants(σ2=σ2,σ3=σ3,σ1=sumsq(tbs)-σ2-σ3)
    σ2 < 0 && return Invariants(σ3=σ3,σ1=σ1,σ2=sumsq(tbs)-σ3-σ1)
end

In [ ]:
λ(x,y,z) = x^2+y^2+z^2-2x*y-2y*z-2z*x
# 
p1mod(σs::Invariants,tbs::ThreeBodySystem) = sqrt(λ(tbs.m0^2,tbs.m1^2,σs.σ1))/(2tbs.m0)
p2mod(σs::Invariants,tbs::ThreeBodySystem) = sqrt(λ(tbs.m0^2,tbs.m2^2,σs.σ2))/(2tbs.m0)
p3mod(σs::Invariants,tbs::ThreeBodySystem) = sqrt(λ(tbs.m0^2,tbs.m3^2,σs.σ3))/(2tbs.m0)
#
function cosθh12(σs::Invariants,tbs::ThreeBodySystem)
    @unpack σ1,σ2,σ3 = σs
    @unpack m0,m1,m2,m3 = tbs
    return (2m0^2*(m1^2+m2^2-σ3)+(m0^2+m1^2-σ1)*(m0^2+m2^2-σ2)) / sqrt(λ(m0^2,m1^2,σ1)*λ(m0^2,m2^2,σ2))
end
function cosθh23(σs::Invariants,tbs::ThreeBodySystem)
    @unpack σ1,σ2,σ3 = σs
    @unpack m0,m1,m2,m3 = tbs
    return (2m0^2*(m2^2+m3^2-σ1)+(m0^2+m2^2-σ2)*(m0^2+m3^2-σ3)) / sqrt(λ(m0^2,m2^2,σ2)*λ(m0^2,m3^2,σ3))
end
function cosθh31(σs::Invariants,tbs::ThreeBodySystem) =
    @unpack σ1,σ2,σ3 = σs
    @unpack m0,m1,m2,m3 = tbs
    return (2m0^2*(m3^2+m1^2-σ2)+(m0^2+m3^2-σ3)*(m0^2+m1^2-σ1)) / sqrt(λ(m0^2,m3^2,σ3)*λ(m0^2,m1^2,σ1))
end
#
Kibble(σs::Invariants,tbs::ThreeBodySystem) = λ(λ(tbs.m0^2,tbs.m1^2,σs.σ1),
                                                λ(tbs.m0^2,tbs.m2^2,σs.σ2),
                                                λ(tbs.m0^2,tbs.m3^2,σs.σ3))
#
σ2of1(z, σ1; tbs::ThreeBodySystem) = tbs.m3^2 + tbs.m1^2 +
    (σ1+tbs.m3^2-tbs.m2^2)*(tbs.m0^2-tbs.m1^2-σ1)/(2σ1) -
    sqrt(λ(tbs.m0^2,tbs.m1^2,σ1)*λ(tbs.m2^2,tbs.m3^2,σ1))/(2σ1)*z

In [ ]:
tbs = ThreeBodySystem(m0=1.1, m1=0.3, m2=0.3, m3=0.3)
@manipulate for σ1 in range((tbs.m2+tbs.m3)^2, (tbs.m0-tbs.m1)^2, length=500),
                σ2 in range((tbs.m3+tbs.m1)^2, (tbs.m0-tbs.m2)^2, length=500)
    σs = Invariants(tbs,σ1=σ1,σ2=σ2)
    (σs,α) = Kibble(σs,tbs) < 0 ? (σs,1.0) : (Invariants(tbs,σ1=0.7^2,σ2=0.7^2), 0.0)
    #
    plot(layout=grid(1,2), size=(900,350))
    plot!(sp=2, xaxis=false, yaxis=false, aspect_ratio=1)
    #
    _p1mod = p1mod(σs,tbs)
    plot!(sp=2, [0,-_p1mod], [0, 0], lab="", arrow=true, lw=2, α=α)
    #
    _p2mod = p2mod(σs,tbs)
    _cosθh12 = cosθh12(σs,tbs)
    _sinθh12 = sqrt(1-_cosθh12^2)
    plot!(sp=2, [0,-_p2mod*_cosθh12], [0,_p2mod*_sinθh12], lab="", arrow=true, lw=2, α=α, seriescolor=2)
    plot!(sp=2, [0,-_p2mod*_cosθh12].-_p1mod, [0,_p2mod*_sinθh12], lab="", arrow=true, lw=0.5, ls=:dash, α=α, seriescolor=2)
    #
    _p3mod = p3mod(σs,tbs)
    _cosθh31 = cosθh31(σs,tbs)
    _sinθh31 = sqrt(1-_cosθh31^2)
    plot!(sp=2, [0,-_p3mod*_cosθh31], [0,-_p3mod*_sinθh31], lab="", arrow=true, lw=2, α=α, seriescolor=3)
    plot!(sp=2, [0,-_p3mod*_cosθh31].-_p1mod .-_p2mod*_cosθh12,
        [0,-_p3mod*_sinθh31].+_p2mod*_sinθh12, lab="", arrow=true, lw=0.5, ls=:dash, α=α, seriescolor=3)
    #
    scatter!(sp=1, [σ1], [σ2], lab="", ms=10,
            xlim=((tbs.m2+tbs.m3)^2, (tbs.m0-tbs.m1)^2),
            ylim=((tbs.m3+tbs.m1)^2, (tbs.m0-tbs.m2)^2))
    #
    σ1v = range((tbs.m2+tbs.m3)^2, (tbs.m0-tbs.m1)^2, length=100)
    σ2v⁺ = σ2of1.(1,σ1v; tbs=tbs)
    σ2v⁻ = σ2of1.(-1,σ1v; tbs=tbs)
    plot!(sp=1, σ1v, [σ2v⁺ σ2v⁻], lab="", l=(:gray))
end